# 重新实现之前的线性热模型
应该只需要读取第一次实验的数据，这里主要想规范一下之前的写法

In [22]:
import pandas as pd
import numpy as np
import os
from keys import *
from utils_thermal_model_raw_process import *
import math
from tqdm import tqdm
from loader import Loader
from thermal_model.data import ThermalModelData

# 读取数据并且缓存成单独pickle

In [23]:
sss = ThermalModelData().load()

   Reading data from cache: ../.cache\thermal_model_data_1101


In [24]:
sss

,Unnamed: 0,date_time,stack_voltage,stack_current,产氢量,产氢累计量,lye_flow,temp_in,sys_pressure,temp_o,...,露点,微氧量,出罐压力,进罐温度,进罐压力,temp_out,delta_temp,cell_voltage,current_density,ambt_temp
0,0,2021-09-24 09:20:00,0.009181,0.275432,-9999.000000,-9999.00000,0.0,21.887623,0.168050,22.034521,...,-79.992653,0.009181,-9999.000000,-9999.000000,-9999.000000,22.328315,0.000000,0.000270,0.648075,28.039091
1,1,2021-09-24 09:20:20,0.009181,0.275432,-9999.000000,-9999.00000,0.0,22.181417,0.168050,22.034521,...,-79.992653,0.009181,-9999.000000,-9999.000000,-9999.000000,22.328315,0.000000,0.000270,0.648075,28.045239
2,2,2021-09-24 09:20:40,0.009181,0.275432,-9999.000000,-9999.00000,0.0,22.181417,0.167267,22.034521,...,-79.992653,0.009181,-9999.000000,-9999.000000,-9999.000000,22.401763,0.073448,0.000270,0.648075,28.051385
3,3,2021-09-24 09:21:00,0.009181,0.275432,-9999.000000,-9999.00000,0.0,21.887623,0.168050,22.034521,...,-79.992653,0.009181,-9999.000000,-9999.000000,-9999.000000,22.401763,0.000000,0.000270,0.648075,28.057527
4,4,2021-09-24 09:21:20,0.009181,0.275432,-9999.000000,-9999.00000,0.0,22.181417,0.167267,22.034521,...,-79.992653,0.009181,-9999.000000,-9999.000000,-9999.000000,22.328315,-0.073448,0.000270,0.648075,28.063666
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1887,1887,2021-12-02 18:59:00,1.591382,3.672420,0.000000,668.63623,0.0,43.628353,0.204089,46.419392,...,-79.992653,0.009181,2.298935,10.811607,16.961685,46.419392,-0.073446,0.046805,8.640989,8.730434
1888,1888,2021-12-02 18:59:20,1.517934,4.406904,0.011001,668.63623,0.0,43.040764,0.203697,46.419392,...,-79.992653,0.009181,2.300159,11.115189,16.961685,46.419392,0.000000,0.044645,10.369186,8.723879
1889,1889,2021-12-02 18:59:40,1.591382,0.000000,0.033003,668.63623,0.0,43.040764,0.203697,46.419392,...,-79.992653,0.009181,2.300159,10.811607,16.961685,46.419392,0.000000,0.046805,0.000000,8.717322
1890,1890,2021-12-02 19:00:00,1.542416,0.000000,0.000000,668.63623,0.0,42.600075,0.204089,46.125595,...,-79.992653,0.009181,2.300159,10.963394,16.961685,45.905252,-0.514139,0.045365,0.000000,8.710762
